In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from tqdm import tqdm
from tqdm import trange

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import ConfusionMatrixDisplay

import random

In [2]:
from AgentBasedModel import *
from AgentBasedModel.extra import *
from AgentBasedModel.visualization import *
from AgentBasedModel.visualization.other import plot_book
from AgentBasedModel.visualization.other import plot_full_book
from AgentBasedModel.visualization.trader import *
from random import randint

In [3]:
more_market_agents=[40, 40, 40, 1]

In [44]:
run utils_nikita//functions.ipynb

# Download data - no LOB

In [74]:
all_data_names = ['huge_data', 'huge_data_no_mm', 'huge_data_undervalued', 'huge_data_undervalued_no_mm']

cur_df = pd.read_csv(f'generated_data/{all_data_names[0]}')

separation_column = []

for i in range(cur_df.shape[0]//250):
    separation_column += [i] * 250

In [75]:
enormous_data = {}

for i in range(4):
    current_division = []
    cur_df = pd.read_csv(f'generated_data/{all_data_names[i]}').drop(['Unnamed: 0'], axis=1)
    cur_df['sep_col'] = separation_column
    
    for j in range(cur_df.shape[0] // 250):
        current_division.append(cur_df[cur_df['sep_col'] == j].reset_index(drop=True).drop(['sep_col'], axis=1))
    
    
    enormous_data[all_data_names[i]] = current_division

In [76]:
huge_data = enormous_data['huge_data']
huge_data_no_mm = enormous_data['huge_data_no_mm']
huge_data_undervalued = enormous_data['huge_data_undervalued']
huge_data_undervalued_no_mm = enormous_data['huge_data_undervalued_no_mm']

all_data = [huge_data, huge_data_no_mm, huge_data_undervalued, huge_data_undervalued_no_mm]

# Download data with LOB

In [5]:
sim_n = 200
iter_n = 250

huge_data_bid = []
huge_data_ask = []

for i in range(sim_n):

    bids_simulation = []
    asks_simulation = []
    
    for j in range(iter_n):
        cur_bid = pd.read_csv(f'data/huge_data/bids/sim_{i}_iter_{j + i * 250}.csv')
        cur_ask = pd.read_csv(f'data/huge_data/asks/sim_{i}_iter_{j + i * 250}.csv')

        bids_simulation.append(cur_bid)
        asks_simulation.append(cur_ask)

    huge_data_bid.append(bids_simulation)
    huge_data_ask.append(asks_simulation)

In [13]:
huge_data_features = pd.read_csv('data/huge_data/huge_data_features.csv')

separation_column = []

for i in range(huge_data_features.shape[0]//250):
    separation_column += [i] * 250

huge_data_features['sep_col'] = separation_column

current_division = []

for j in range(huge_data_features.shape[0] // 250):
    current_division.append(huge_data_features[huge_data_features['sep_col'] == j].reset_index(drop=True).drop(['sep_col'], axis=1))

new_huge_data_features = current_division

# NN classification

In [186]:
# pip install torch

In [187]:
import torch
from torch import nn
from torch.utils.data import DataLoader

## Dataset functions

In [208]:
class trainDataset():
    def __init__(self, data_x, data_y, device="cuda"):
        self.data = data_x
        self.device = device
        self.labels = data_y
        
    def __getitem__(self, x):
        data_x = self.data.iloc[x]
        data_y = [self.labels.iloc[x]]
        
        data = torch.Tensor(data_x.values).to(self.device)
        label = torch.Tensor(data_y).type(torch.LongTensor).to(self.device)
        
        return data, label
    
    def __len__(self):
        return len(self.labels)

In [259]:
class trainDataset_w_LOB():
    def __init__(self, data_x, data_y, min_iteration_indeces, max_iteration_indeces, 
                 data_bid, data_ask, iterations_length=250, device="cuda"):
        
        self.data = data_x
        self.device = device
        self.labels = data_y
        
        self.bids = huge_data_bid
        self.asks = huge_data_ask

        self.min_iteration_indeces = min_iteration_indeces
        self.max_iteration_indeces = max_iteration_indeces

        self.iterations_length = iterations_length
        #[0][min_iteration_indeces: max_iteration_indeces + 1]

        # preparing bid and ask info

        collected_bids = []
        collected_asks = []

        for i in range(len(self.bids)):
            collected_bids = collected_bids + self.bids[i][self.min_iteration_indeces: self.max_iteration_indeces + 1]
            collected_asks = collected_asks + self.asks[i][self.min_iteration_indeces: self.max_iteration_indeces + 1]

        self.bids = collected_bids
        self.asks = collected_asks

        # print(len(self.labels), len(self.bids), len(self.asks))
        
    def __getitem__(self, x):
        data_x = self.data.iloc[x]
        data_y = [self.labels.iloc[x]]

        data_bid = self.bids[x][['price', 'qty']]
        data_ask = self.asks[x][['price', 'qty']]
        
        data = torch.Tensor(data_x.values).to(self.device)
        label = torch.Tensor(data_y).type(torch.LongTensor).to(self.device)

        data_bid = torch.Tensor(data_bid.values[:16]).to(self.device)
        data_ask = torch.Tensor(data_ask.values[:16]).to(self.device)

        data = data.unsqueeze(dim=-1)
        
        result = torch.cat((data.T, data_bid.T, data_ask.T))
        result = result.reshape([result.shape[0] * result.shape[1]])
        
        return result, label
    
    def __len__(self):
        return len(self.labels)

## NN Classifiers architectures

In [260]:
class LSTM_model(nn.Module):
    def __init__(self):
        super(LSTM_model, self).__init__()
        self.lstm = nn.LSTM(17, 30, 3, batch_first=True)
        self.linear = nn.Linear(17, 20)
        self.linear2 = nn.Linear(20, 3)
        self.sigmoid = nn.Sigmoid()
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        
        h_0 = torch.zeros(3, x.size(0), 30).requires_grad_().to(device)
        c_0 = torch.zeros(3, x.size(0), 30).requires_grad_().to(device)
        
        output, (h_index, c_index) = self.lstm(x, (h_0, c_0))
            
        output = self.linear(output)
        
        output = self.relu(output)
        
        output = self.linear2(output)
        
        output = self.sigmoid(output)
        
        return output

In [261]:
class SimpleModel(nn.Module):
    def __init__(self):
        
        super(SimpleModel, self).__init__()
        
        self.linear = nn.Linear(16, 30)
        self.linear2 = nn.Linear(30, 20)
        self.linear3 = nn.Linear(20, 10)
        self.linear4 = nn.Linear(10, 3)
        
#         self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.linear4(x)
        x = self.softmax(x)
        
        return x

In [262]:
class SimpleModel_LOB(nn.Module):
    def __init__(self):
        
        super(SimpleModel_LOB, self).__init__()
        
        self.linear = nn.Linear(80, 160)
        self.linear2 = nn.Linear(160, 100)
        self.linear3 = nn.Linear(100, 50)
        self.linear4 = nn.Linear(50, 20)
        self.linear5 = nn.Linear(20, 3)
        
#         self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.linear4(x)
        x = self.relu(x)
        x = self.linear5(x)
        x = self.softmax(x)
        
        return x

## Training function

In [354]:
def train_nn(model, train_ds, eval_ds, loss_f, n_epoch=1, batch_size=32, learning_rate=0.00001, device='cpu'):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # general recording
    training_loss = []
    evaluation_loss = []

    training_accuracy = []
    evaluation_accuracy = []
    
    for epoch in trange(n_epoch):

        # epoch performance recording
        epoch_train_loss = 0
        epoch_evaluation_loss = 0

        epoch_train_loss = 0
        epoch_evaluation_accuracy = 0

        epoch_evaluation_accuracy_list = []
        
        # training cycle
        
        # data_y = train_df['difference'] + 1
        # data_x =  train_df.drop(['difference'], axis=1)
        # train_ds = trainDataset(data_x, data_y, device=device)

        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False)

        model.train()

        n_of_iterations = 0
        for el in train_dl:
                
            data = el[0]
            target = el[1].squeeze()

            y_pred = model(data)
            
            optimizer.zero_grad()
            
            loss = loss_f(y_pred, target)

            loss.backward()

            optimizer.step()

            epoch_train_loss += loss
            n_of_iterations += 1


            if n_of_iterations == 100:
                break
            
        epoch_train_loss = epoch_train_loss/n_of_iterations

        # testing cycle
        # data_y = eval_df['difference'] + 1
        # data_x =  eval_df.drop(['difference'], axis=1)
        # eval_ds = trainDataset(data_x, data_y, device=device)

        eval_dl = DataLoader(eval_ds, batch_size=data_x.shape[0], shuffle=False)

        model.eval()
        
        for el in eval_dl:
                
            data = el[0]
            target = el[1].squeeze()
    
            y_pred = model(data)
    
            loss = loss_f(y_pred, target)
            epoch_evaluation_loss = loss

            y_classes = torch.argmax(y_pred, dim=1)
            epoch_evaluation_accuracy = accuracy_score(y_classes, target)

            break

        print(f"epoch {epoch} train loss - {epoch_train_loss}, eval loss - {epoch_evaluation_loss}, accuracy - {round(epoch_evaluation_accuracy, 2)}")
        
        training_loss.append(epoch_train_loss)
        evaluation_loss.append(epoch_evaluation_loss)
        epoch_evaluation_accuracy_list.append(round(epoch_evaluation_accuracy, 2))
        

## Training - only collected features

In [355]:
train_df_5 = prepare_huge_data(new_huge_data_features[:160], shift=shift_percentage['huge_data'][1][0], percentage=shift_percentage['huge_data'][1][1]).drop(['future_price', 'index', 'dividends', 'price', 'fundamental_v', 'trajectory_number'], axis=1)
eval_df_5 = prepare_huge_data(new_huge_data_features[160:], shift=shift_percentage['huge_data'][1][0], percentage=shift_percentage['huge_data'][1][1]).drop(['future_price', 'index', 'dividends', 'price', 'fundamental_v', 'trajectory_number'], axis=1)

loss_f = nn.CrossEntropyLoss()

In [356]:
random.seed(2114)

data_y = train_df_5['difference'] + 1
data_x =  train_df_5.drop(['difference', 'Unnamed: 0'], axis=1)
train_ds = trainDataset(data_x, data_y, device=device)

data_y = eval_df_5['difference'] + 1
data_x =  eval_df_5.drop(['difference', 'Unnamed: 0'], axis=1)
eval_ds = trainDataset(data_x, data_y, device=device)

### simple connected nn

In [366]:
random.seed(2114)
torch.manual_seed(2114)
simple_model = SimpleModel()

In [367]:
simple_model.linear.weight[0][0].item()

0.17340821027755737

In [368]:
device='cpu'

In [369]:
train_nn(simple_model, train_ds, eval_ds, loss_f, n_epoch=30, batch_size=32, learning_rate=0.0001, device='cpu')

  3%|█▍                                          | 1/30 [00:01<00:49,  1.71s/it]

epoch 0 train loss - 1.1139522790908813, eval loss - 1.1134015321731567, accuracy - 0.32


  7%|██▉                                         | 2/30 [00:03<00:45,  1.63s/it]

epoch 1 train loss - 1.1074010133743286, eval loss - 1.1081417798995972, accuracy - 0.33


 10%|████▍                                       | 3/30 [00:04<00:43,  1.60s/it]

...

epoch 27 train loss - 1.082740306854248, eval loss - 1.0811930894851685, accuracy - 0.47


 97%|█████████████████████████████████████████▌ | 29/30 [00:52<00:01,  1.93s/it]

epoch 28 train loss - 1.0814313888549805, eval loss - 1.0800400972366333, accuracy - 0.47


100%|███████████████████████████████████████████| 30/30 [00:53<00:00,  1.80s/it]

epoch 29 train loss - 1.0800795555114746, eval loss - 1.0788003206253052, accuracy - 0.47


## Training - features + LOB

In [370]:
train_df_5 = prepare_huge_data(new_huge_data_features[:160], shift=shift_percentage['huge_data'][1][0], percentage=shift_percentage['huge_data'][1][1]).drop(['future_price', 'index', 'dividends', 'price', 'fundamental_v', 'trajectory_number'], axis=1)
eval_df_5 = prepare_huge_data(new_huge_data_features[160:], shift=shift_percentage['huge_data'][1][0], percentage=shift_percentage['huge_data'][1][1]).drop(['future_price', 'index', 'dividends', 'price', 'fundamental_v', 'trajectory_number'], axis=1)

loss_f = nn.CrossEntropyLoss()

In [371]:
iteration_indeces = train_df_5['Unnamed: 0'].unique()

min_iteration_indeces = min(iteration_indeces)
max_iteration_indeces = max(iteration_indeces)

In [372]:
train_ds = trainDataset_w_LOB(train_df_5.drop(['Unnamed: 0', 'difference'], axis=1), 
                                  train_df_5['difference'] + 1, min_iteration_indeces, max_iteration_indeces, 
                                  huge_data_bid, huge_data_ask, device='cpu')

eval_ds = trainDataset_w_LOB(eval_df_5.drop(['Unnamed: 0', 'difference'], axis=1), 
                                  eval_df_5['difference'] + 1, min_iteration_indeces, max_iteration_indeces, 
                                  huge_data_bid, huge_data_ask, device='cpu')

### simple connected nn

In [383]:
random.seed(2114)
torch.manual_seed(2114)
simple_model = SimpleModel_LOB()

In [384]:
device='cpu'

In [385]:
train_nn(simple_model, train_ds, eval_ds, loss_f, n_epoch=10, batch_size=32, learning_rate=0.00001, device='cpu')

 10%|████▍                                       | 1/10 [00:14<02:09, 14.36s/it]

epoch 0 train loss - 1.1190379858016968, eval loss - 1.1002893447875977, accuracy - 0.35


 20%|████████▊                                   | 2/10 [00:28<01:55, 14.47s/it]

epoch 1 train loss - 1.1008983850479126, eval loss - 1.1006940603256226, accuracy - 0.35


 30%|█████████████▏                              | 3/10 [00:43<01:41, 14.47s/it]

...

epoch 7 train loss - 1.0958770513534546, eval loss - 1.1004284620285034, accuracy - 0.35


 90%|███████████████████████████████████████▌    | 9/10 [02:11<00:14, 14.87s/it]

epoch 8 train loss - 1.095334529876709, eval loss - 1.1003360748291016, accuracy - 0.35


100%|███████████████████████████████████████████| 10/10 [02:26<00:00, 14.63s/it]

epoch 9 train loss - 1.0948565006256104, eval loss - 1.1003025770187378, accuracy - 0.34
